In [51]:
# this is the result
# ac 0.879

In [52]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from stn import spatial_transformer_network as transformer
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import GaussianNoise
import numpy as np
import matplotlib.pyplot as plt
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import cv2



In [53]:
train = ImageDataGenerator(rescale=1 / 255,validation_split=0.175)
test = ImageDataGenerator(rescale=1 / 255)

In [54]:
image_size=(150,150)

In [55]:
train_dataset = train.flow_from_directory("../database/sharpness1reduce/train",
                                          target_size=image_size,
                                          class_mode="categorical",
                                          subset="training",
                                          
                                         )
valid_dataset = train.flow_from_directory("../database/sharpness1reduce/train",
                                          target_size=image_size,
                                          class_mode="categorical",                                          
                                          subset="validation")

test_dataset = test.flow_from_directory("../database/sharpness1reduce/test",
                                        target_size=image_size,                                       
                                        class_mode="categorical")

Found 3389 images belonging to 49 classes.
Found 688 images belonging to 49 classes.
Found 1919 images belonging to 49 classes.


In [56]:
model = tf.keras.models.Sequential()
    
model.add(tf.keras.Input(shape=(150, 150, 3)))
model.add(tf.keras.layers.Conv2D(32,(7, 7),padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(32,(5, 5),padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(32,(3, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3 ,3),strides=2))

model.add(GaussianNoise(0.01))
        
model.add(tf.keras.layers.Conv2D(64,(3, 3), padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(64,(3, 3), padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(64,(3, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3 ,3)))

model.add(GaussianNoise(0.01))
        
model.add(tf.keras.layers.Conv2D(128,(3, 3),padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(128,(3, 3),padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(128,(3, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(4, 4)))

model.add(GaussianNoise(0.01))

model.add(tf.keras.layers.Conv2D(256,(3, 3), padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(256,(3, 3), padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(256,(3, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(4 ,4)))

model.add(GaussianNoise(0.01))

model.add(tf.keras.layers.Conv2D(512,(3, 3), padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(512,(3, 3), padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Conv2D(512,(3, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2 ,2),strides=1))
model.add(GaussianNoise(0.01))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(1024,kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(GaussianNoise(0.1))
model.add(Activation("relu"))          
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(512,kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(GaussianNoise(0.1))
model.add(Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))
        
model.add(tf.keras.layers.Dense(256,kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(GaussianNoise(0.1))
model.add(Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))
        
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(49, activation="softmax"))
model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add] name: conv2d_126/kernel/Initializer/random_uniform/

In [ ]:

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_dataset, steps_per_epoch=40,validation_data=valid_dataset, epochs=250,shuffle=True)


score = model.evaluate(test_dataset, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("echo this is the end ***")
